<a href="https://colab.research.google.com/github/Kaldah/Projet-Apprentissage/blob/main/MineRL_v1_0_0_VPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation etc etc (takes ~20 minutes)

In [ ]:
!sudo add-apt-repository -y ppa:openjdk-r/ppa > /dev/null
!sudo apt-get purge openjdk-* -y > /dev/null
!sudo apt-get install openjdk-8-jdk -y > /dev/null

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 16.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [ ]:
!git clone https://github.com/openai/video-pre-training
%cd video-pre-training

Cloning into 'video-pre-training'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 137 (delta 36), reused 24 (delta 17), pack-reused 83
Receiving objects: 100% (137/137), 94.38 KiB | 7.86 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/video-pre-training


In [ ]:
!pip3 install --upgrade git+https://github.com/minerllabs/minerl@v1.0.0
import minerl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/minerllabs/minerl (to revision v1.0.0) to /tmp/pip-req-build-jjcdbbwl
  Running command git clone -q https://github.com/minerllabs/minerl /tmp/pip-req-build-jjcdbbwl
  Running command git checkout -b v1.0.0 --track origin/v1.0.0
  Switched to a new branch 'v1.0.0'
  Branch 'v1.0.0' set up to track remote branch 'v1.0.0' from 'origin'.
     |████████████████████████████████| 793 kB 16.8 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 6.4 MB 53.6 MB/s 
     |████████████████████████████████| 281 kB 59.2 MB/s 
     |████████████████████████████████| 89 kB 9.7 MB/s 
     |████████████████████████████████| 46 kB 4.2 MB/s 
     |████████████████████████████████| 3.1 MB 30.7 MB/s 
     |████████████████████████████████| 812 kB 58.4 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |███████████████

In [ ]:
!pip3 install pyvirtualdisplay gym -q
!pip3 install -U colabgymrender -q
!sudo apt install xvfb > /dev/null
!pip install gym3 imageio-ffmpeg -q

# Set up environment (rerun this if you get startup errors)

In [ ]:
from minerl.herobraine.env_specs.human_survival_specs import HumanSurvival as Env
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display

RuntimeError: ignored

In [ ]:
display = Display(visible=0, size=(720, 480))
display.start()
env = Env(resolution=[640, 360], fov_range=[70., 70.], guiscale_range=[1., 1.]).make()
env = Recorder(env, "./video", fps=30)

# Set up model

In [ ]:
from agent import MineRLAgent
import pickle
import torch


weights_file = "https://openaipublic.blob.core.windows.net/minecraft-rl/models/rl-from-early-game-2x.weights"  #@param {type: "string", allow-input:true} ["https://openaipublic.blob.core.windows.net/minecraft-rl/models/foundation-model-1x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/foundation-model-2x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/foundation-model-1x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/bc-house-3x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/bc-early-game-2x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/bc-early-game-3x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/rl-from-foundation-2x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/rl-from-house-2x.weights", "https://openaipublic.blob.core.windows.net/minecraft-rl/models/rl-from-early-game-2x.weights"]
multiplier = [x for x in ["1x", "2x", "3x"] if x in weights_file][0]
!wget -c {weights_file} -O model
!wget -c https://openaipublic.blob.core.windows.net/minecraft-rl/models/{multiplier}.model -O model
!wget -c {weights_file} -O weights
agent_parameters = pickle.load(open("model", "rb"))
policy_kwargs = agent_parameters["model"]["args"]["net"]["args"]
pi_head_kwargs = agent_parameters["model"]["args"]["pi_head_opts"]
pi_head_kwargs["temperature"] = float(pi_head_kwargs["temperature"])
agent = MineRLAgent(env, policy_kwargs=policy_kwargs, pi_head_kwargs=pi_head_kwargs, device="cuda" if torch.cuda.is_available() else "cpu")
agent.load_weights("weights")

# Run model

In [ ]:
obs = env.reset()

In [ ]:
from IPython.display import clear_output
from matplotlib import pyplot as plt
from tqdm.auto import trange


play_steps = 922  #@param {type: "number"}
live_display = False  #@param {type: "boolean"}
try:
    for _ in trange(play_steps):  # The t part will get erased anyway
        minerl_action = agent.get_action(obs)
        obs, reward, done, info = env.step(minerl_action)
        if done:
            break
        if live_display:
            clear_output(wait=True)
            plt.axis("off")
            plt.imshow(obs["pov"])
            plt.show()
except KeyboardInterrupt:
    pass


In [ ]:
#@title Show the result
env.release()
env.play()

by @nev#4950